In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [11]:
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay, RocCurveDisplay
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from mysql.connector import connect
from getpass import getpass

In [12]:
db = connect(
             host = 'localhost',
             user = 'root',
             passwd = getpass('enter your password please'),
             database = 'sakila'
             )

In [13]:
query = 'SELECT * FROM subset_data'

In [105]:
df = pd.read_sql(query,db)
df

/var/folders/6g/z7c4_v2s4x5g_4nmtry98b2c0000gn/T/ipykernel_17579/2118518691.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,db)


,film_id,release_year,language_id,rental_duration,rental_rate,length,rating,special_features,rental_date
0,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-07
1,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-08
2,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-08
3,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-05
4,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-06
...,...,...,...,...,...,...,...,...,...
16040,1000,2006,1,3,4.99,50,NC-17,"{Behind the Scenes, Trailers, Commentaries}",2005-05
16041,1000,2006,1,3,4.99,50,NC-17,"{Behind the Scenes, Trailers, Commentaries}",2005-06
16042,1000,2006,1,3,4.99,50,NC-17,"{Behind the Scenes, Trailers, Commentaries}",2005-07
16043,1000,2006,1,3,4.99,50,NC-17,"{Behind the Scenes, Trailers, Commentaries}",2005-07


In [106]:
df.drop_duplicates
df

,film_id,release_year,language_id,rental_duration,rental_rate,length,rating,special_features,rental_date
0,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-07
1,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-08
2,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-08
3,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-05
4,1,2006,1,6,0.99,86,PG,"{Deleted Scenes, Behind the Scenes}",2005-06
...,...,...,...,...,...,...,...,...,...
16040,1000,2006,1,3,4.99,50,NC-17,"{Behind the Scenes, Trailers, Commentaries}",2005-05
16041,1000,2006,1,3,4.99,50,NC-17,"{Behind the Scenes, Trailers, Commentaries}",2005-06
16042,1000,2006,1,3,4.99,50,NC-17,"{Behind the Scenes, Trailers, Commentaries}",2005-07
16043,1000,2006,1,3,4.99,50,NC-17,"{Behind the Scenes, Trailers, Commentaries}",2005-07


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16045 entries, 0 to 16044
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           16045 non-null  int64  
 1   release_year      16045 non-null  int64  
 2   language_id       16045 non-null  int64  
 3   rental_duration   16045 non-null  int64  
 4   rental_rate       16045 non-null  float64
 5   length            16045 non-null  int64  
 6   rating            16045 non-null  object 
 7   special_features  16045 non-null  object 
 8   rental_date       16045 non-null  object 
dtypes: float64(1), int64(5), object(3)
memory usage: 1.1+ MB


In [108]:
df.rental_date.value_counts()

rental_date
2005-07    6709
2005-08    5686
2005-06    2311
2005-05    1156
2006-02     182
2024-02       1
Name: count, dtype: int64

In [109]:
df.rental_date = df.rental_date.apply(lambda x: 1 if x=='2005-08' else 0)

In [110]:
df.rental_date

0        0
1        1
2        1
3        0
4        0
        ..
16040    0
16041    0
16042    0
16043    0
16044    1
Name: rental_date, Length: 16045, dtype: int64

In [111]:
df[['release_year','language_id']].describe()

,release_year,language_id
count,16045.0,16045.0
mean,2006.0,1.0
std,0.0,0.0
min,2006.0,1.0
25%,2006.0,1.0
50%,2006.0,1.0
75%,2006.0,1.0
max,2006.0,1.0


In [112]:
df = df.drop(columns=['release_year','language_id','special_features'])

In [113]:
df

,film_id,rental_duration,rental_rate,length,rating,rental_date
0,1,6,0.99,86,PG,0
1,1,6,0.99,86,PG,1
2,1,6,0.99,86,PG,1
3,1,6,0.99,86,PG,0
4,1,6,0.99,86,PG,0
...,...,...,...,...,...,...
16040,1000,3,4.99,50,NC-17,0
16041,1000,3,4.99,50,NC-17,0
16042,1000,3,4.99,50,NC-17,0
16043,1000,3,4.99,50,NC-17,0


In [114]:
df.rating.value_counts()

rating
PG-13    3585
NC-17    3293
PG       3213
R        3181
G        2773
Name: count, dtype: int64

In [115]:
df.rating = df.rating.replace({'PG-13':0, 'NC-17':1, 'PG':2, 'R':3, 'G':4})

In [117]:
y = df.rental_date
X = df.drop('rental_date', axis=1)

In [120]:
numeric_features = ['film_id', 'rental_duration','rental_rate','length']
categorical_features = ["rating"]

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25)

In [122]:
numeric_transformer = Pipeline(steps = [("imputer", SimpleImputer(strategy="mean")),
                                       ("scaler", StandardScaler())
                                       ])

In [123]:
categorical_transformer = OneHotEncoder(drop = "first")

In [124]:
preprocessor = ColumnTransformer(
            transformers = [("numerical", numeric_transformer, numeric_features), 
                            ("categorical", categorical_transformer, categorical_features)]
                            )

In [125]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [48]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [126]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

In [127]:
pred_train = classifier.predict(X_train)
display(pred_train)

pred_test = classifier.predict(X_test)
display(pred_test)

array([0, 0, 0, ..., 0, 0, 0])

array([0, 0, 0, ..., 0, 0, 0])

In [129]:
prob_train = classifier.predict_proba(X_train)
display(prob_train)

prob_test = classifier.predict_proba(X_test)
display(prob_test)

array([[0.63343559, 0.36656441],
       [0.63177042, 0.36822958],
       [0.65688227, 0.34311773],
       ...,
       [0.63965794, 0.36034206],
       [0.65708304, 0.34291696],
       [0.66005694, 0.33994306]])

array([[0.64350122, 0.35649878],
       [0.65598286, 0.34401714],
       [0.65219046, 0.34780954],
       ...,
       [0.63402676, 0.36597324],
       [0.65628169, 0.34371831],
       [0.65250896, 0.34749104]])

In [130]:
print("accuracy on train set", accuracy_score(y_train, pred_train))
print("accuracy on test set", accuracy_score(y_test, pred_test))
print("f1 score on train set", f1_score(y_train, pred_train))
print("f1 score on test set", f1_score(y_test, pred_test))

accuracy on train set 0.647635668578077
accuracy on test set 0.6395812562313061
f1 score on train set 0.0
f1 score on test set 0.0
